In [ ]:
!mv kaggle.json /root/.kaggle/
!kaggle competitions download -c jpx-tokyo-stock-exchange-prediction
!unzip ./jpx-tokyo-stock-exchange-prediction.zip -d jpx-tokyo-stock-exchange-prediction
!rm -r sample_data

In [15]:
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
class time2vec(nn.Module):
  def __init__(self, in_features, out_features):
    super().__init__()
    self.w_linear = nn.Parameter(data=torch.rand(in_features, 1))
    self.b_linear = nn.Parameter(data=torch.rand(1))
    self.w_function = nn.Parameter(data=torch.rand(in_features, out_features-1))
    self.b_function = nn.Parameter(data=torch.rand(out_features-1))

    #maybe a bit more straightforward
    #self.linear_params = nn.Linear(in_features, 1, bias=True)
    #self.function_params = nn.Linear(in_features, out_features-1, bias=True)

    #initialize params?
    #self.w_linear.init.kaiming_normal_()
    #self.b_linear.init.kaiming_normal_()
    #self.w_function.init.kaiming_normal_()
    #self.b_function.init.kaiming_normal_()
  def forward(self, x):
    linear_out = torch.matmul(x, self.w_linear)+self.b_linear
    func_out = torch.sin(torch.matmul(x, self.w_function)+self.b_function)
    return torch.concat((linear_out, func_out), dim=-1)

In [ ]:
class TSTransformer(nn.Module):
  def __init__(self, in_features, time_features=1, mlp_dim=1024, enc_layers=2, enc_heads=2):
    super().__init__()
    self.time2vec = time2vec(in_features, time_features)
    self.encoder_layer = nn.TransformerEncoderLayer(d_model=in_features+time_features, nhead=enc_heads, 
                                                    dropout=0, activation=F.gelu, batch_first=True, norm_first=False)
    self.encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=enc_layers)

    self.mlp = nn.Linear(in_features+time_features, mlp_dim)
    self.regressor = nn.Linear(mlp_dim, 1)

  def forward(self, x):
    time_embeddings = self.time2vec(x)
    x = torch.concat((x, time_embeddings), dim=-1)
    x = self.encoder(x)

    x = F.relu(self.mlp(x))
    x = self.regressor(x)

    return x

In [87]:
#getting rid of null values for this experiment
dframe = pd.read_csv('jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv', parse_dates=['Date'])
dframe_1 = dframe[~dframe['Close'].isnull()]
dframe_1

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1,20170104_1332,2017-01-04,1332,568.0,576.0,563.0,571.0,2798500,1.0,NaN,False,0.012324
2,20170104_1333,2017-01-04,1333,3150.0,3210.0,3140.0,3210.0,270800,1.0,NaN,False,0.006154
3,20170104_1376,2017-01-04,1376,1510.0,1550.0,1510.0,1550.0,11300,1.0,NaN,False,0.011053
4,20170104_1377,2017-01-04,1377,3270.0,3350.0,3270.0,3330.0,150800,1.0,NaN,False,0.003026
...,...,...,...,...,...,...,...,...,...,...,...,...
2332526,20211203_9990,2021-12-03,9990,514.0,528.0,513.0,528.0,44200,1.0,NaN,False,0.034816
2332527,20211203_9991,2021-12-03,9991,782.0,794.0,782.0,794.0,35900,1.0,NaN,False,0.025478
2332528,20211203_9993,2021-12-03,9993,1690.0,1690.0,1645.0,1645.0,7200,1.0,NaN,False,-0.004302
2332529,20211203_9994,2021-12-03,9994,2388.0,2396.0,2380.0,2389.0,6500,1.0,NaN,False,0.009098


In [89]:
#extracting trading data from only one company 
dframe_1301 = dframe_1[dframe_1['SecuritiesCode'] == 1301]
dframe_1301

,RowId,Date,SecuritiesCode,Open,High,Low,Close,Volume,AdjustmentFactor,ExpectedDividend,SupervisionFlag,Target
0,20170104_1301,2017-01-04,1301,2734.0,2755.0,2730.0,2742.0,31400,1.0,NaN,False,0.000730
1865,20170105_1301,2017-01-05,1301,2743.0,2747.0,2735.0,2738.0,17900,1.0,NaN,False,0.002920
3730,20170106_1301,2017-01-06,1301,2734.0,2744.0,2720.0,2740.0,19900,1.0,NaN,False,-0.001092
5595,20170110_1301,2017-01-10,1301,2745.0,2754.0,2735.0,2748.0,24200,1.0,NaN,False,-0.005100
7460,20170111_1301,2017-01-11,1301,2748.0,2752.0,2737.0,2745.0,9300,1.0,NaN,False,-0.003295
...,...,...,...,...,...,...,...,...,...,...,...,...
2322531,20211129_1301,2021-11-29,1301,2975.0,2984.0,2951.0,2951.0,14300,1.0,NaN,False,0.003793
2324531,20211130_1301,2021-11-30,1301,2953.0,2997.0,2900.0,2900.0,20500,1.0,NaN,False,0.007558
2326531,20211201_1301,2021-12-01,1301,2909.0,2936.0,2909.0,2911.0,10600,1.0,NaN,False,0.016706
2328531,20211202_1301,2021-12-02,1301,2949.0,2973.0,2933.0,2933.0,15200,1.0,NaN,False,-0.003689


In [105]:
#Getting a quick batch
train_batch = dframe_1301[:128][['Open', 'High', 'Low', 'Close', 'Volume']]
target_batch = dframe_1301[:128]['Target']

train_batch = torch.tensor(train_batch.values, dtype=torch.float32).unsqueeze(0)
target_batch = torch.tensor(target_batch.values, dtype=torch.float32).unsqueeze(0).unsqueeze(-1)

In [140]:
#Model runs on a single batch
model = TSTransformer(in_features = 5)

crit = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr = 0.0000001)

for epoch in range(100):
  out = model(train_batch)
  loss = crit(out, target_batch)
  loss.backward()
  optim.step()

  print(loss)

tensor(0.0170, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0169, grad_fn=<MseLossBackward0>)
tensor(0.0168, grad_fn=<MseLossBackward0>)
tensor(0.0168, grad_fn=<MseLossBackward0>)
tensor(0.0168, grad_fn=<MseLossBackward0>)
tensor(0.0168, grad_fn=<MseLossBackward0>)
tensor(0.0168, grad_fn=<MseLossBackward0>)
tensor(0.0168, grad_fn=<MseLossBackward0>)
tensor(0.0168, grad_fn=<MseLossBackward0>)
tensor(0.0167, grad_fn=<MseLossBackward0>)
tensor(0.0167, grad_fn=<MseLossBackward0>)
tensor(0.0167, grad_fn=<MseLossBackward0>)
tensor(0.0167, grad_fn=<MseLossBackward0>)
tensor(0.0167, grad_fn=<MseLossBackward0>)
tensor(0.0167, grad_fn=<MseLossBackward0>)
tensor(0.0166, grad_fn=<MseLossBackward0>)
tensor(0.01